In [18]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
There is no [mac64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/96.0.4664.35/chromedriver_mac64.zip
Driver has been saved in cache [/Users/liang/.wdm/drivers/chromedriver/mac64/96.0.4664.35]


In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
# One is that we're searching for elements with a specific combination of tag (div) and attribute (list_text). 
#As an example, ul.item_list would be found in HTML as <ul class="item_list">.
#Secondly, we're also telling our browser to wait one second before searching for components. 
#The optional delay is useful because sometimes dynamic pages take a little while to load, 
#especially if they are image-heavy.

browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [5]:
#we'll set up the HTML parser
#Notice how we've assigned slide_elem as the variable to look for the <div /> tag and its descendent 
#(the other tags within the <div /> element)? This is our parent element. This means that this element 
#holds all of the other elements within it, and we'll reference it when we want to filter search results 
#even further. The . is used for selecting classes, such as list_text, so the code 'div.list_text' pinpoints 
#the <div /> tag with the class of list_text. CSS works from right to left, such as returning the last item on 
#the list instead of the first. Because of this, when using select_one, the first matching element returned 
#will be a <li /> element with a class of slide and all nested elements within it.
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [6]:
#In this line of code, we chained .find onto our previously assigned variable, slide_elem. 
#When we do this, we're saying, "This variable holds a ton of information, so look inside of that information to 
#find this specific data." The data we're looking for is the content title, which we've specified by saying, 
#"The specific data is in a <div /> with a class of 'content_title'.
slide_elem.find('div', class_='content_title')


<div class="content_title">NASA's Perseverance Rover Is Midway to Mars </div>

In [7]:
# Use the parent element to find the first `a` tag and save it as `news_title`
#to remove the html and just get the title text
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Perseverance Rover Is Midway to Mars "

In [8]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination."

### Featured Images

In [13]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [14]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()
#Notice the indexing chained at the end of the first line of code? With this, we've stipulated that we 
#want our browser to click the second button. [1]


In [15]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')
#With the new page loaded onto our automated browser, it needs to be parsed so we can continue and scrape 
#the full-size image URL

In [16]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [17]:
# Use the base URL to create an absolute URL
#This variable holds our f-string.
#This is an f-string, a type of string formatting used for print statements in Python.
#The curly brackets hold a variable that will be inserted into the f-string when it's executed.
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [19]:
#df = pd.read_htmldf = pd.read_html('https://galaxyfacts-mars.com')[0] With this line, we're creating a new 
#DataFrame from the HTML table. The Pandas function read_html() specifically searches for and returns a list of 
#tables found in the HTML. By specifying an index of 0, we're telling Pandas to pull only the first table it 
#encounters, or the first item in the list. Then, it turns the table into a DataFrame.
#df.columns=['description', 'Mars', 'Earth'] Here, we assign columns to the new DataFrame for additional clarity.
#df.set_index('description', inplace=True) By using the .set_index() function, we're turning the 
#Description column into the DataFrame's index. inplace=True means that the updated index will remain in place, 
#without having to reassign the DataFrame to a new variable.
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [22]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [23]:
browser.quit()